In [1]:
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

In [2]:
%pwd

'd:\\Projekt\\notebooks'

In [3]:
%cd ..

d:\Projekt


d:\Projekt\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from src.prep_vid_v2 import get_images_tensor
import pyarrow.parquet as pq
from pathlib import Path
from tqdm import tqdm
from src.model import load_model
import torch
import pandas as pd

d:\Projekt\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Projekt\venv\Lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start set

In [5]:
data_path = [Path('data/datasets/eval'), Path('data/datasets/test')]
models = ['adaface', 'arcface']
# models = ['adaface']

In [6]:
for path in data_path:
    for model_name in models:
        dataset = path.parts[-1]
        vids = list(path.rglob('*.mp4'))
        data = {'video_id': [], 'person_id': [], 'embedding': [], 'frame_id': []}
        model = load_model(model_name)
        for video in tqdm(vids, desc=f'Model: {model_name}, Dataset: {dataset}'):
            video_id = video.stem
            person_id = video.parts[-2]
            images = get_images_tensor(video)
            if images is None or images.shape == torch.Size([0]):
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(None)
                data['frame_id'].append(None)
                # print(f'No faces detected in {video}')
                continue
            embeddings = model(images.float().cuda())
            if isinstance(embeddings, torch.Tensor):
                embeddings = embeddings.detach().numpy()
            for i, embedding in enumerate(embeddings):
                embedding = embedding.squeeze()
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(embedding)
                data['frame_id'].append(int(i))
        df = pd.DataFrame(data)
        df.to_parquet(f'data/fusion_win/{model_name}_{dataset}.parquet')



d:\Projekt\models\AdaFace\inference.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  statedict = torch.load(adaface_models[architecture])['state_dict']
Model: adaface, D

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: arcface, Dataset: eval:   0%|          | 0/2487 [00:00<?, ?it/s]d:\Projekt\src\model.py:45: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  return torch.tensor(emb)
Model: arcface, Dataset: eval: 100%|██████████| 2487/2487 [54:09<00:00,  1.31s/it]  
d:\Projekt\models\AdaFace\inference.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `Tru

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: arcface, Dataset: test: 100%|██████████| 2476/2476 [56:25<00:00,  1.37s/it]  


In [7]:
df[(df['video_id'] == '94') & (df['person_id'] == 'id07802')]

,video_id,person_id,embedding,frame_id
6938,94,id07802,"[0.6601988, -0.45184615, -0.3119703, 0.4264250...",0.0
6939,94,id07802,"[1.1444043, -1.0436709, -0.11216894, 1.1115494...",1.0
6940,94,id07802,"[0.8238619, -0.2899797, -0.2755927, 0.46496618...",2.0
6941,94,id07802,"[0.59772307, -1.4290673, 0.0387201, 0.6771873,...",3.0


In [8]:
path = 'data/datasets/train/'
data_path = [Path(path+str(i)) for i in range(13)]

In [9]:

for path in data_path:
    for model_name in models:
        dataset = path.parts[-2]
        split = path.parts[-1]
        vids = list(path.rglob('*.mp4'))
        data = {'video_id': [], 'person_id': [], 'embedding': [], 'frame_id': []}
        model = load_model(model_name)
        for video in tqdm(vids, desc=f'Model: {model_name}, Dataset: {dataset}'):
            video_id = video.stem
            person_id = video.parts[-2]
            images = get_images_tensor(video)
            if images is None or images.shape == torch.Size([0]):
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(None)
                data['frame_id'].append(None)
                # print(f'No faces detected in {video}')
                continue
            embeddings = model(images.float().cuda())
            if isinstance(embeddings, torch.Tensor):
                embeddings = embeddings.detach().numpy()
            for i, embedding in enumerate(embeddings):
                embedding = embedding.squeeze()
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(embedding)
                data['frame_id'].append(int(i))
        df = pd.DataFrame(data)
        df.to_parquet(f'data/fusion_win/{model_name}_{dataset}_{split}.parquet')

d:\Projekt\models\AdaFace\inference.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  statedict = torch.load(adaface_models[architecture])['state_dict']
Model: adaface, D

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: adaface, Dataset: train: 100%|██████████| 4500/4500 [1:15:52<00:00,  1.01s/it]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: adaface, Dataset: train: 100%|██████████| 4273/4273 [1:22:58<00:00,  1.17s/it]  


...

Model: adaface, Dataset: train: 100%|██████████| 4423/4423 [1:24:31<00:00,  1.15s/it]  


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: adaface, Dataset: train: 100%|██████████| 766/766 [17:17<00:00,  1.35s/it]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: arcface, Dataset: train: 100%|██████████| 766/766 [19:35<00:00,  1.53s/it]
